In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.entry_processed as entry_processed

#add the parent folder to sys path so we can import from the notebook subfolder
sys.path.insert(0,'..')

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/nf_demo_short -n 2'
args = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full", "-n", "2"]

pxs, embedding, categories, classavg, palette = entry_processed.read_processed(args)





In [ ]:
import xfmkit.geopixeio as geopixeio

geopixeio.export_regions(categories, pxs.dimensions)